<a href="https://colab.research.google.com/github/ananyakbcodes/CROP_PEST_DETECTION/blob/PRANITA/Preprocessing%20the%20data%20set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'plantdisease' dataset.
Path to dataset files: /kaggle/input/plantdisease


In [2]:
# Options:
#  - Set USE_KAGGLEHUB = True to use kagglehub (automated).
#  - If that fails or you prefer, use Kaggle API (requires kaggle.json upload).
USE_KAGGLEHUB = True
DATASET_SLUG = "emmarex/plantdisease"   # the dataset you shared

dataset_root = None

if USE_KAGGLEHUB:
    try:
        import kagglehub
        print("Using kagglehub to download dataset:", DATASET_SLUG)
        dataset_root = kagglehub.dataset_download(DATASET_SLUG)
        print("kagglehub returned:", dataset_root)
    except Exception as e:
        print("kagglehub failed:", e)
        print("Falling back to Kaggle API method. Set USE_KAGGLEHUB=False to skip fallback.")
        USE_KAGGLEHUB = False

if not USE_KAGGLEHUB:
    print("\n-- Kaggle API fallback --")
    print("If you haven't uploaded kaggle.json yet, you'll be prompted. Follow the instructions.")
    # create directory and instruct user to upload kaggle.json via Colab file picker
    from google.colab import files
    kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")
    if not os.path.exists(kaggle_json_path):
        print("Please upload your kaggle.json file (Kaggle → Account → Create API Token).")
        uploaded = files.upload()
        if "kaggle.json" in uploaded:
            os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
            open(kaggle_json_path, "wb").write(uploaded["kaggle.json"])
            os.chmod(kaggle_json_path, 0o600)
            print("kaggle.json uploaded to ~/.kaggle/kaggle.json")
        else:
            print("No kaggle.json uploaded. Please upload and re-run this cell.")
            raise SystemExit("kaggle.json required for Kaggle API fallback.")
    # now use kaggle API to download
    print("Downloading dataset via Kaggle API...")
    !kaggle datasets download -d {DATASET_SLUG} -p "{RAW_DOWNLOAD_DIR}"
    # find downloaded zip file and unzip
    zips = [p for p in Path(RAW_DOWNLOAD_DIR).glob("*.zip")]
    if len(zips)==0:
        print("No zip found in", RAW_DOWNLOAD_DIR, "- maybe the dataset was already unzipped or name differs.")
    else:
        for z in zips:
            print("Unzipping:", z)
            with zipfile.ZipFile(z, 'r') as zip_ref:
                zip_ref.extractall(RAW_DOWNLOAD_DIR)
        print("Unzipped all zips in", RAW_DOWNLOAD_DIR)
    # set dataset_root to the main extracted folder (try common names)
    # We'll attempt to find the folder containing images
    for candidate in ["PlantVillage", "plantdisease", "Plant_Disease", "images", "plantdisease-dataset"]:
        cand_path = os.path.join(RAW_DOWNLOAD_DIR, candidate)
        if os.path.exists(cand_path):
            dataset_root = cand_path
            break
    # if still none, pick the largest dir under RAW_DOWNLOAD_DIR
    if dataset_root is None:
        children = [p for p in Path(RAW_DOWNLOAD_DIR).iterdir() if p.is_dir()]
        if children:
            children_sorted = sorted(children, key=lambda p: sum(1 for _ in p.rglob("*")), reverse=True)
            dataset_root = str(children_sorted[0])
    print("Detected dataset_root:", dataset_root)

# Final check
if dataset_root is None:
    print("ERROR: Could not determine dataset root automatically. Please set dataset_root to the folder containing class subfolders.")
else:
    print("Dataset root:", dataset_root)


Using kagglehub to download dataset: emmarex/plantdisease
Using Colab cache for faster access to the 'plantdisease' dataset.
kagglehub returned: /kaggle/input/plantdisease
Dataset root: /kaggle/input/plantdisease


In [5]:
# Try to detect the folder that contains class subfolders (folders that contain many images)
from pathlib import Path

def find_image_root(base_dir):
    base = Path(base_dir)
    candidates = []
    for child in base.rglob("*"):
        if child.is_dir():
            # count images inside
            count = sum(1 for f in child.glob("*") if f.suffix.lower() in [".jpg",".jpeg",".png"])
            # candidate if it contains subdirectories or many images
            if count>0 and any(c.is_dir() for c in child.iterdir()):
                candidates.append((child, count))
    # fallback: top-level dirs with many images
    if not candidates:
        for child in base.iterdir():
            if child.is_dir():
                count = sum(1 for f in child.rglob("*") if f.suffix.lower() in [".jpg",".jpeg",".png"])
                candidates.append((child, count))
    if candidates:
        candidates_sorted = sorted(candidates, key=lambda x: x[1], reverse=True)
        return str(candidates_sorted[0][0])
    return None

if 'dataset_root' in globals() and dataset_root:
    image_root = find_image_root(dataset_root)
else:
    image_root = None

print("Auto-detected image root:", image_root)
print("If this is incorrect, set RAW_IMAGE_DIR manually to the folder that contains class subfolders.")
# If detection failed, user should set manually
RAW_IMAGE_DIR = image_root or input("Enter path to RAW image folder (e.g. /content/crop_project/raw_download/PlantVillage): ")

Auto-detected image root: /kaggle/input/plantdisease/PlantVillage
If this is incorrect, set RAW_IMAGE_DIR manually to the folder that contains class subfolders.


In [9]:
# Preprocess images: robust, tolerant to errors
IMG_SIZE = (224,224)
TRAIN_SPLIT = 0.8
MAX_PER_CLASS = 200   # set to None to use all images. Lower value keeps repo small.
SEED = 42

from pathlib import Path
import random
from tqdm import tqdm
random.seed(SEED)

def prepare_images(raw_dir, out_dir, img_size=(224,224), train_split=0.8, max_per_class=None, seed=42):
    raw_dir = Path(raw_dir)
    out_dir = Path(out_dir)
    train_root = out_dir / "train"
    test_root = out_dir / "test"
    train_root.mkdir(parents=True, exist_ok=True)
    test_root.mkdir(parents=True, exist_ok=True)
    classes = sorted([p.name for p in raw_dir.iterdir() if p.is_dir()])
    if not classes:
        raise ValueError(f"No class subfolders found in {raw_dir}. Each class should be a folder containing images.")
    print(f"Found {len(classes)} classes. Example classes: {classes[:8]}")
    summary = {}
    for cls in classes:
        src = raw_dir / cls
        imgs = [p for p in src.rglob("*") if p.suffix.lower() in [".jpg",".jpeg",".png"]]
        if not imgs:
            print(f"Warning: No images found for class {cls} at {src}")
            summary[cls] = {"train":0,"test":0}
            continue
        imgs = sorted(imgs)
        if max_per_class:
            imgs = imgs[:max_per_class]
        random.shuffle(imgs)
        n_train = int(len(imgs)*train_split)
        train_imgs = imgs[:n_train]
        test_imgs = imgs[n_train:]
        # create class folders
        (train_root/cls).mkdir(parents=True, exist_ok=True)
        (test_root/cls).mkdir(exist_ok=True)
        # process and save
        cnt_train = 0
        cnt_test = 0
        for p in tqdm(train_imgs, desc=f"Train {cls}", leave=False):
            try:
                im = Image.open(p).convert("RGB")
                im = im.resize(img_size, Image.BILINEAR)
                out_path = train_root/cls/p.name
                im.save(out_path, optimize=True, quality=85)
                cnt_train += 1
            except Exception as e:
                print("skip train", p, ":", e)
        for p in tqdm(test_imgs, desc=f"Test {cls}", leave=False):
            try:
                im = Image.open(p).convert("RGB")
                im = im.resize(img_size, Image.BILINEAR)
                out_path = test_root/cls/p.name
                im.save(out_path, optimize=True, quality=85)
                cnt_test += 1
            except Exception as e:
                print("skip test", p, ":", e)
        summary[cls] = {"train":cnt_train, "test":cnt_test}
    return summary

# Run preprocessing (this can take time)
print("Running preprocessing...")
summary = prepare_images(RAW_IMAGE_DIR, CLEAN_IMG_DIR, img_size=IMG_SIZE, train_split=TRAIN_SPLIT, max_per_class=MAX_PER_CLASS, seed=SEED)
print("Preprocessing done.")

Running preprocessing...
Found 15 classes. Example classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight']


skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/11b85334-0029-430e-873d-9402bd9e5e9f___JR_B.Spot 9031.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/3042800d-559d-4250-bb62-23c346259685___NREC_B.Spot 1980.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/184221a3-0f0e-4acc-b5be-7b853349cd0d___JR_B.Spot 3286.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/313c8e04-9195-4b11-9f63-72ce704f78f3___NREC_B.Spot 9245.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/1b0cfb07-f452-49e0-85ad-45f3f519ca7a___JR_B.Spot 9094.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/1d35e088-7147-4bac-9cce-9330a3f5906d___NREC_B.Spot 9096.JPG : name 'Im

skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/15f3b9e4-9127-4404-904b-173dbe878fd6___JR_B.Spot 3226.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/2e400707-ea59-4355-889a-32d5e56c1e1c___NREC_B.Spot 1936.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/0bd0f439-013b-40ed-a6d1-4e67e971d437___JR_B.Spot 3272.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/2b8c82d0-2e5d-483a-93eb-53192fa486c9___NREC_B.Spot 1858.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/162a3375-31e6-48f4-9b78-ab35054fb86c___JR_B.Spot 3397.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/2fd2f664-70aa-40d6-b770-bb839944ce18___JR_B.Spot 3261.JPG : name 'Image' is 

skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/0eeb924f-88db-44e6-a278-e016fa2d25a4___JR_HL 8016.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/118ec2cf-48d0-4bb9-9487-612ea61471a1___JR_HL 7730.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/1bf8d046-2b1c-44bf-9a00-9f16e0971d2a___JR_HL 8281.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/1d6e01ff-affd-4867-895d-fb82b6b0dce3___JR_HL 7952.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/18570bd8-1b60-4773-b618-ef34fe3f602f___JR_HL 5915.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/116a8c0e-51fb-4205-ace6-87865b2329d5___JR_HL 8657.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/P

skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/24711a2b-ec05-461e-913c-6cc65ec3b87e___JR_HL 8648.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/174172a7-b8c3-4afc-aa95-637c08472656___JR_HL 7923.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/0e526c7b-72ee-4547-916b-5cba013097b6___JR_HL 7735.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/0c102dad-d31f-469b-89d1-6fe7bf93708c___JR_HL 7689.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/18f75aea-62fb-4c71-998a-ff62d078ab21___JR_HL 8174.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/1e6e9501-4510-4950-abea-36882683aa5c___JR_HL 8435.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__

...

skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/05d890dc-e8a2-4a7d-b19c-5ca629327502___YLCV_GCREC 2827.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/0273f811-11bf-4fc3-b8b1-fe33ac4b3692___UF.GRC_YLCV_Lab 03226.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/05e34d35-6039-44e8-919b-fe1eb112c331___UF.GRC_YLCV_Lab 02325.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/0464962a-0f2c-45bc-855a-06bfe656577a___UF.GRC_YLCV_Lab 01858.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/065a4b60-c606-4dae-b255-144d3442221e___UF.GRC_YLCV_Lab 02597.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/09f

skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/71aa5741-6923-4593-9a23-bc7f58dc40b2___PSU_CG 2183.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/67662638-1bfa-4ff6-8305-2d8544277ef0___PSU_CG 2222.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/4e35d3db-e33d-493d-80f2-6a3fdce77959___PSU_CG 2135.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/79894594-feb4-46ca-830c-5f1cf5b11063___PSU_CG 2063.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/09910ece-e505-4e19-ae9b-0d9e2eaa2cb1___PSU_CG 2047.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/503c6a57-2af3-48ed-b96f-e2cbf6fb5dd1___PSU_CG 2263.JPG : name 'Image' is not defined
skip train /kagg

skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/7ddb3878-8899-42bd-8ec1-5ee012f30cf2___PSU_CG 2188.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/146eaf68-7f61-4717-ac81-2f8bb4533fec___PSU_CG 2406.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/75c566a5-8c20-4294-8138-4f06a3c2df7c___PSU_CG 2158.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/2370fa79-6f15-4ae5-a3cb-2d585a0043c2___PSU_CG 2178.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/69726f1b-4d1c-4647-aef2-090e57246501___PSU_CG 2131.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/3ecbb7b8-ad66-4944-886e-de2ae2c2c35e___PSU_CG 2095.JPG : name 'Image' is not defined
skip test /kaggle/inpu

skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/07cb5416-a029-467f-a306-252c2a64d2e6___RS_HL 0543.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1d10291e-a1ce-4309-b6a4-d034319e3875___GH_HL Leaf 251.1.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/185440e0-ca0a-4cd6-aba7-d03fae21552f___RS_HL 0141.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1af0bfe1-4bcf-4b8b-be66-5d0953eb647e___GH_HL Leaf 482.2.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1a308834-fdd7-4121-ae03-5736d8d52e26___RS_HL 9689.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/05598cc1-60b9-4436-a233-973c42eff2d6___GH_HL Leaf 503.1.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/17cffb66-bb7f-43

skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/196e5fd8-8b4e-4a45-9db7-ae59c877e9a7___GH_HL Leaf 284.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1dffd343-4634-4b33-95f4-eecc755aec8f___GH_HL Leaf 168.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/0bfe260c-a71e-474e-bfb1-78f1e5ca5698___RS_HL 0497.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/02b4afdf-e1de-4c0e-a38d-3f19afeb9ea9___RS_HL 0493.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/087e3b12-a80b-435f-8973-a3b07888f8b4___RS_HL 9847.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1ddf5950-e27f-4928-8f4e-33242953e306___RS_HL 0181.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/050e7bff-2f2d-4c92-9422-05ce93f1d

In [8]:
# Define the directory to save cleaned images
CLEAN_IMG_DIR = "/kaggle/working/plant_images/cleaned"

In [10]:
# Preprocess images: robust, tolerant to errors
IMG_SIZE = (224,224)
TRAIN_SPLIT = 0.8
MAX_PER_CLASS = 200   # set to None to use all images. Lower value keeps repo small.
SEED = 42

from pathlib import Path
random.seed(SEED)

def prepare_images(raw_dir, out_dir, img_size=(224,224), train_split=0.8, max_per_class=None, seed=42):
    raw_dir = Path(raw_dir)
    out_dir = Path(out_dir)
    train_root = out_dir / "train"
    test_root = out_dir / "test"
    train_root.mkdir(parents=True, exist_ok=True)
    test_root.mkdir(parents=True, exist_ok=True)
    classes = sorted([p.name for p in raw_dir.iterdir() if p.is_dir()])
    if not classes:
        raise ValueError(f"No class subfolders found in {raw_dir}. Each class should be a folder containing images.")
    print(f"Found {len(classes)} classes. Example classes: {classes[:8]}")
    summary = {}
    for cls in classes:
        src = raw_dir / cls
        imgs = [p for p in src.rglob("*") if p.suffix.lower() in [".jpg",".jpeg",".png"]]
        if not imgs:
            print(f"Warning: No images found for class {cls} at {src}")
            summary[cls] = {"train":0,"test":0}
            continue
        imgs = sorted(imgs)
        if max_per_class:
            imgs = imgs[:max_per_class]
        random.shuffle(imgs)
        n_train = int(len(imgs)*train_split)
        train_imgs = imgs[:n_train]
        test_imgs = imgs[n_train:]
        # create class folders
        (train_root/cls).mkdir(parents=True, exist_ok=True)
        (test_root/cls).mkdir(parents=True, exist_ok=True)
        # process and save
        cnt_train = 0
        cnt_test = 0
        for p in tqdm(train_imgs, desc=f"Train {cls}", leave=False):
            try:
                im = Image.open(p).convert("RGB")
                im = im.resize(img_size, Image.BILINEAR)
                out_path = train_root/cls/p.name
                im.save(out_path, optimize=True, quality=85)
                cnt_train += 1
            except Exception as e:
                print("skip train", p, ":", e)
        for p in tqdm(test_imgs, desc=f"Test {cls}", leave=False):
            try:
                im = Image.open(p).convert("RGB")
                im = im.resize(img_size, Image.BILINEAR)
                out_path = test_root/cls/p.name
                im.save(out_path, optimize=True, quality=85)
                cnt_test += 1
            except Exception as e:
                print("skip test", p, ":", e)
        summary[cls] = {"train":cnt_train, "test":cnt_test}
    return summary

# Run preprocessing (this can take time)
print("Running preprocessing...")
summary = prepare_images(RAW_IMAGE_DIR, CLEAN_IMG_DIR, img_size=IMG_SIZE, train_split=TRAIN_SPLIT, max_per_class=MAX_PER_CLASS, seed=SEED)
print("Preprocessing done.")


Running preprocessing...
Found 15 classes. Example classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight']


skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/11b85334-0029-430e-873d-9402bd9e5e9f___JR_B.Spot 9031.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/3042800d-559d-4250-bb62-23c346259685___NREC_B.Spot 1980.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/184221a3-0f0e-4acc-b5be-7b853349cd0d___JR_B.Spot 3286.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/313c8e04-9195-4b11-9f63-72ce704f78f3___NREC_B.Spot 9245.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/1b0cfb07-f452-49e0-85ad-45f3f519ca7a___JR_B.Spot 9094.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/1d35e088-7147-4bac-9cce-9330a3f5906d___NREC_B.Spot 9096.JPG : name 'Im

skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/15f3b9e4-9127-4404-904b-173dbe878fd6___JR_B.Spot 3226.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/2e400707-ea59-4355-889a-32d5e56c1e1c___NREC_B.Spot 1936.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/0bd0f439-013b-40ed-a6d1-4e67e971d437___JR_B.Spot 3272.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/2b8c82d0-2e5d-483a-93eb-53192fa486c9___NREC_B.Spot 1858.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/162a3375-31e6-48f4-9b78-ab35054fb86c___JR_B.Spot 3397.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___Bacterial_spot/2fd2f664-70aa-40d6-b770-bb839944ce18___JR_B.Spot 3261.JPG : name 'Image' is 

skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/0eeb924f-88db-44e6-a278-e016fa2d25a4___JR_HL 8016.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/118ec2cf-48d0-4bb9-9487-612ea61471a1___JR_HL 7730.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/1bf8d046-2b1c-44bf-9a00-9f16e0971d2a___JR_HL 8281.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/1d6e01ff-affd-4867-895d-fb82b6b0dce3___JR_HL 7952.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/18570bd8-1b60-4773-b618-ef34fe3f602f___JR_HL 5915.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/116a8c0e-51fb-4205-ace6-87865b2329d5___JR_HL 8657.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/P

skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/24711a2b-ec05-461e-913c-6cc65ec3b87e___JR_HL 8648.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/174172a7-b8c3-4afc-aa95-637c08472656___JR_HL 7923.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/0e526c7b-72ee-4547-916b-5cba013097b6___JR_HL 7735.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/0c102dad-d31f-469b-89d1-6fe7bf93708c___JR_HL 7689.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/18f75aea-62fb-4c71-998a-ff62d078ab21___JR_HL 8174.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__bell___healthy/1e6e9501-4510-4950-abea-36882683aa5c___JR_HL 8435.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Pepper__

...

skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/05d890dc-e8a2-4a7d-b19c-5ca629327502___YLCV_GCREC 2827.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/0273f811-11bf-4fc3-b8b1-fe33ac4b3692___UF.GRC_YLCV_Lab 03226.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/05e34d35-6039-44e8-919b-fe1eb112c331___UF.GRC_YLCV_Lab 02325.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/0464962a-0f2c-45bc-855a-06bfe656577a___UF.GRC_YLCV_Lab 01858.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/065a4b60-c606-4dae-b255-144d3442221e___UF.GRC_YLCV_Lab 02597.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus/09f

skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/71aa5741-6923-4593-9a23-bc7f58dc40b2___PSU_CG 2183.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/67662638-1bfa-4ff6-8305-2d8544277ef0___PSU_CG 2222.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/4e35d3db-e33d-493d-80f2-6a3fdce77959___PSU_CG 2135.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/79894594-feb4-46ca-830c-5f1cf5b11063___PSU_CG 2063.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/09910ece-e505-4e19-ae9b-0d9e2eaa2cb1___PSU_CG 2047.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/503c6a57-2af3-48ed-b96f-e2cbf6fb5dd1___PSU_CG 2263.JPG : name 'Image' is not defined
skip train /kagg

skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/7ddb3878-8899-42bd-8ec1-5ee012f30cf2___PSU_CG 2188.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/146eaf68-7f61-4717-ac81-2f8bb4533fec___PSU_CG 2406.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/75c566a5-8c20-4294-8138-4f06a3c2df7c___PSU_CG 2158.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/2370fa79-6f15-4ae5-a3cb-2d585a0043c2___PSU_CG 2178.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/69726f1b-4d1c-4647-aef2-090e57246501___PSU_CG 2131.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato__Tomato_mosaic_virus/3ecbb7b8-ad66-4944-886e-de2ae2c2c35e___PSU_CG 2095.JPG : name 'Image' is not defined
skip test /kaggle/inpu

skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/07cb5416-a029-467f-a306-252c2a64d2e6___RS_HL 0543.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1d10291e-a1ce-4309-b6a4-d034319e3875___GH_HL Leaf 251.1.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/185440e0-ca0a-4cd6-aba7-d03fae21552f___RS_HL 0141.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1af0bfe1-4bcf-4b8b-be66-5d0953eb647e___GH_HL Leaf 482.2.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1a308834-fdd7-4121-ae03-5736d8d52e26___RS_HL 9689.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/05598cc1-60b9-4436-a233-973c42eff2d6___GH_HL Leaf 503.1.JPG : name 'Image' is not defined
skip train /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/17cffb66-bb7f-43

skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/196e5fd8-8b4e-4a45-9db7-ae59c877e9a7___GH_HL Leaf 284.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1dffd343-4634-4b33-95f4-eecc755aec8f___GH_HL Leaf 168.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/0bfe260c-a71e-474e-bfb1-78f1e5ca5698___RS_HL 0497.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/02b4afdf-e1de-4c0e-a38d-3f19afeb9ea9___RS_HL 0493.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/087e3b12-a80b-435f-8973-a3b07888f8b4___RS_HL 9847.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/1ddf5950-e27f-4928-8f4e-33242953e306___RS_HL 0181.JPG : name 'Image' is not defined
skip test /kaggle/input/plantdisease/PlantVillage/Tomato_healthy/050e7bff-2f2d-4c92-9422-05ce93f1d

In [11]:
# Show summary stats
total_train = sum(v['train'] for v in summary.values())
total_test  = sum(v['test'] for v in summary.values())
print(f"Total train images: {total_train}")
print(f"Total test images:  {total_test}")
print("\nTop classes (train/test counts):")
for k,v in sorted(summary.items(), key=lambda x: x[0])[:30]:
    print(f"{k}: train={v['train']}, test={v['test']}")


Total train images: 0
Total test images:  0

Top classes (train/test counts):
Pepper__bell___Bacterial_spot: train=0, test=0
Pepper__bell___healthy: train=0, test=0
Potato___Early_blight: train=0, test=0
Potato___Late_blight: train=0, test=0
Potato___healthy: train=0, test=0
Tomato_Bacterial_spot: train=0, test=0
Tomato_Early_blight: train=0, test=0
Tomato_Late_blight: train=0, test=0
Tomato_Leaf_Mold: train=0, test=0
Tomato_Septoria_leaf_spot: train=0, test=0
Tomato_Spider_mites_Two_spotted_spider_mite: train=0, test=0
Tomato__Target_Spot: train=0, test=0
Tomato__Tomato_YellowLeaf__Curl_Virus: train=0, test=0
Tomato__Tomato_mosaic_virus: train=0, test=0
Tomato_healthy: train=0, test=0


In [12]:
# Display up to 6 sample images from different classes (works in Colab)
from IPython.display import display
sample_dirs = sorted([d for d in Path(CLEAN_IMG_DIR, "train").iterdir() if d.is_dir()])[:6]
print("Sample class folders:", [d.name for d in sample_dirs])
for d in sample_dirs:
    imgs = list(d.glob("*"))
    if not imgs:
        continue
    img_path = imgs[0]
    display(Image.open(img_path).resize((256,256)))
    print("Class:", d.name, "->", img_path.name)


Sample class folders: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot']


In [14]:
# Create sample zip with up to N images per class from train (so repo remains light)
import os
import zipfile
from pathlib import Path

SAMPLE_ZIP_PATH = os.path.join(PROJECT_ROOT, "data_images_sample.zip")
MAX_PER_CLASS_FOR_ZIP = 20   # change as desired

with zipfile.ZipFile(SAMPLE_ZIP_PATH, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    for cls_dir in Path(CLEAN_IMG_DIR, "train").iterdir():
        if not cls_dir.is_dir():
            continue
        imgs = sorted([p for p in cls_dir.glob("*") if p.suffix.lower() in [".jpg",".jpeg",".png"]])[:MAX_PER_CLASS_FOR_ZIP]
        for p in imgs:
            arcname = os.path.join("data", "images", "train", cls_dir.name, p.name)
            zf.write(p, arcname)
print("Created sample zip:", SAMPLE_ZIP_PATH, "size (MB):", os.path.getsize(SAMPLE_ZIP_PATH)/1024/1024)

NameError: name 'PROJECT_ROOT' is not defined

In [16]:
# Define the directory to save cleaned images
CLEAN_IMG_DIR = "/kaggle/working/plant_images/cleaned"

In [15]:
# Define the project root directory
PROJECT_ROOT = "/kaggle/working"

In [18]:
# Optional: Save processed images to Google Drive for persistence/sharing
from google.colab import drive
import os
import shutil
drive.mount('/content/drive', force_remount=True)

DEST_ON_DRIVE = os.path.join("/content/drive/MyDrive", "crop_project", "data", "images")
print("Copying to Drive:", DEST_ON_DRIVE)
if os.path.exists(DEST_ON_DRIVE):
    print("Destination exists on Drive - skipping copy to avoid overwriting. Remove the folder manually if you want to replace.")
else:
    shutil.copytree(CLEAN_IMG_DIR, DEST_ON_DRIVE)
    print("Copied processed images to Drive.")

Mounted at /content/drive
Copying to Drive: /content/drive/MyDrive/crop_project/data/images
Copied processed images to Drive.


In [19]:
# If you want to push the sample to GitHub from your laptop, download it locally
from google.colab import files
files.download(SAMPLE_ZIP_PATH)


NameError: name 'SAMPLE_ZIP_PATH' is not defined

In [21]:
# Define sample zip path
import os
SAMPLE_ZIP_PATH = os.path.join(PROJECT_ROOT, "data_images_sample.zip")